In [ ]:
import pandas as pd
import numpy as np

# read the data input 
dfInput = pd.read_csv('https://raw.githubusercontent.com/budapestpy-workshops/sample_files/master/workshop5.csv', delimiter=',')
dfInput.head()

In [ ]:
#Read the country data 
dfCountry = pd.read_csv('https://raw.githubusercontent.com/budapestpy-workshops/sample_files/master//Country.csv', delimiter=',')
dfCountry.head()

In [ ]:
#setting the data dimension to show

set1Filter = 'SH.XPD.CHEX.PC.CD'
set1Label = 'Current health expenditure per capita (current US$)'
#set2Filter = 'SH.DYN.NCOM.ZS'
#set2Label = 'Mortality from CVD, cancer, diabetes or CRD between exact ages 30 and 70 (%)'
set2Filter = 'SP.DYN.LE00.IN'
set2Label = 'Life expectancy at birth, total (years)'


In [ ]:
# preparing the data frame to store the data in the right format to show
dfData = pd.DataFrame(columns = ['Country Code','Set1','Set2','Year'])
dfData.head()

In [ ]:
# filling the data frame from the  input data
for i in range(1960,2019):
    yearFilter = str(i)

    dsSet1 = dfInput[(dfInput['Indicator Code'] == set1Filter)][['Country Code', yearFilter]]
    dsSet1.rename(columns={yearFilter:'Set1'}, inplace = True)

    dsSet2 = dfInput[(dfInput['Indicator Code'] == set2Filter)][['Country Code', yearFilter]]
    dsSet2.rename(columns={yearFilter:'Set2'}, inplace = True)

    dfWork = pd.merge(dsSet1,dsSet2, how = 'inner', on = ['Country Code'])
    dfWork['Year'] = yearFilter

    dfData = dfData.append(dfWork, ignore_index = True, sort = False)

dfData.dropna(inplace = True)
dfData = pd.merge(dfData, dfCountry, how = 'inner', on = ['Country Code'])
dfData.head()

In [ ]:
dfData.info()

In [ ]:
# defining the ranges from the data set
set1Range = dfData['Set1'].agg([np.min, np.max])
set2Range = dfData['Set2'].agg([np.min, np.max])
yearRange = dfData['Year'].agg([np.min, np.max])
set1Range['amin'] = -100
yearRange = yearRange.astype('int')

print('Set1 min: {}  max:{}'.format(set1Range['amin'], set1Range['amax']))
print('Set2 min: {}  max:{}'.format(set2Range['amin'], set2Range['amax']))
print('Year min: {}  max:{}'.format(yearRange['amin'], yearRange['amax']))


In [ ]:
from bokeh.io import output_notebook,show
from bokeh.plotting import figure
from bokeh.layouts import widgetbox, column
from bokeh.models import CustomJS, ColumnDataSource, Slider
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral7

#output_notebook()
output_notebook()

# setting the data source of the diagram
source1 = ColumnDataSource(dfData[dfData['Year'] == '2000'])
source = ColumnDataSource(dfData)
factors = dfData['Region'].unique()

# diagram setting
p = figure(plot_width=900, plot_height=400, title = set2Label + ' by ' + set1Label,
          x_axis_label= set1Label, y_axis_label = set2Label,
          x_range = set1Range, y_range = set2Range)

# drawing the circles
p.circle(x='Set1', y='Set2', size=12, alpha=0.7, source = source1, 
         fill_color=factor_cmap('Region', palette=Spectral7, factors = factors),legend = 'Region')

# setting the legend attributes 
p.legend.title = 'Regions'
p.legend.location = 'bottom_right'

show(p)

In [ ]:
# create the slider
slider = Slider( title = 'Year', start = yearRange['amin'], end=yearRange['amax'], step = 1, value = yearRange['amin'])
# add the slider to a widgetbox
layout = column(widgetbox(slider),p)


In [ ]:
# some js code for the magic 
   
sliderChange = CustomJS(args=dict(source1=source1, source=source, slider=slider), code="""
    var data1 = source1.data;
    var data = source.data;
    var f = slider.value;

    data1['Set1'] = [];
    data1['Set2'] = [];
    data1['Year'] = [];

    console.log(data1);
    for (i in data['Year']) {
        if (data['Year'][i] == slider.value){
            console.log(data['Year'][i]);
            data1['Set1'].push(data['Set1'][i]);
            data1['Set2'].push(data['Set2'][i]);
            data1['Year'].push(data['Year'][i]);
        }
        
    }
    
    // necessary becasue we mutated source.data in-place
    source1.change.emit();
""")

slider.js_on_change('value', sliderChange)


In [ ]:
show(layout)